<style>
pre {
 white-space: pre-wrap !important;
}
.table-striped > tbody > tr:nth-of-type(odd) {
    background-color: #f9f9f9;
}
.table-striped > tbody > tr:nth-of-type(even) {
    background-color: white;
}
.table-striped td, .table-striped th, .table-striped tr {
    border: 1px solid black;
    border-collapse: collapse;
    margin: 1em 2em;
}
.rendered_html td, .rendered_html th {
    text-align: left;
    vertical-align: middle;
    padding: 4px;
}
</style>

# I/O Kung-Fu: get your data in and out of [Vaex](https://github.com/vaexio/vaex)

## Data input

Every project starts with reading in some data. Vaex supports several data sources:

- Binary file formats:
 
     - [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format#HDF5)
     - [Apache Arrow](https://arrow.apache.org/)
     - [Apache Parquet](https://parquet.apache.org/)
     - [FITS](https://en.wikipedia.org/wiki/FITS)
     
 - Text based file formats:
 
     - [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
     - [ASCII](https://en.wikipedia.org/wiki/Text_file)
     - [JSON](https://www.json.org/json-en.html)
     
 - In-memory data representations:
 
     - [panads](https://pandas.pydata.org/) DataFrames and everything that pandas can read
     - [Apache Arrow](https://arrow.apache.org/) Tables
     - [numpy](https://numpy.org/) arrays
     - Python dictionaries
     - Single row DataFrames
     
The following examples show the best practices of getting your data in Vaex.

### Binary file formats

If your data is already in one of the supported binary file formats (HDF5, Apache Arrow, Apache Parquet, FITS), opening it with Vaex rather simple:

```
import vaex 

df_1 = vaex.open('./my_data/my_file_1.hdf5')
df_2 = vaex.open('./my_data/my_file_1.arrow')
df_3 = vaex.open('./my_data/my_file_1.parquet')
df_4 = vaex.open('./my_data/my_file_1.fits')
```

Opening such data is instantenous regardless of the file size on disk: Vaex will just memory-map the data instead of reading it in memory. This is the optimal way of working with large datasets that are larger than available RAM.

If your data is contained within multiple files, one can open them all simultaneously like this:

```
df = vaex.open('./my_data/my_file*.hdf5')
# alternatively
df = vaex.open_many(['./my_data/my_file_1.hdf5', './my_data/my_file_2.hdf5', './my_data/my_file_2.hdf5'])
```
The result will be a single DataFrame object containing all of the data coming from all files.

The data does not necessarily have to be local. With Vaex you can open a HDF5 file straight from Amazon's S3:

```
df = vaex.open('s3://vaex/taxi/yellow_taxi_2009_2015_f32.hdf5?anon=true')
```

In this case the data will be lazily downloaded and cached to the local machine. "Lazily downloaded" means that Vaex will only download the portions of the data you really need. For example: imagine that we have a file hosted on S3 that has 100 columns and 1 billion rows. Getting a preview of the DataFrame via `print(df)` for instance will download only the first and last 5 rows. If we than proceed to make calculations or plots with only 5 columns, only the data from those columns will be downloaded and caches to the local machine. 

By default, data that is streamed from S3 is cached at ` $HOME/.vaex/file-cache/s3`, and thus successive access is as fast as native disk access. One can also use the `profile_name` argument to use a specific S3 profile, which will than be passed to `s3fs.core.S3FileSystem`.

### Text based file formats

Datasets are still commonly stored in text-based file formats such as CSV. Since text-based file formats are not memory-mappable, they have to be read in memory. If the contents of a CSV file fits into the available RAM, one can simply do:

```
df = vaex.from_csv('./my_data/my_file.csv')
# or alternatively 
df = vaex.read_csv('./my_data/my_file.csv')  # `vaex.read_csv` is an alias to `vaex.from_csv`
```

Vaex is using pandas for reading CSV files in the background, so one can pass any arguments to the `vaex.from_csv` or `vaex.read_csv` as one would pass to `pandas.read_csv` and specify for example separators, column names and column types. In addition to this, if you specify the `convert=True` argument, the data will be automatically converted to an HDF5 file format behind the scenes, thus freeing RAM and allowing you to work with your data in a memory-efficient, out-of-core manner.

If the CSV file is so large that it can not fit into RAM all at one time, one can convert the data to HDF5 simply by:

```
df = vaex.from_csv('./my_data/my_big_file.csv', convert=True, chunk_size=5_000_000)
```

When the above line is executed, Vaex will read the CSV in chunks, and convert each chunk to a temporary HDF5 file on disk. All temporary files are then concatenated into a single HDF5 file, and the temporary files deleted. The size of the individual chunks to be read can be specified via the `chunk_size` argument. Note that this automatic conversion requires free disk space of twice the final HDF5 file size.

It is also common the data to be stored in JSON files. To read such data in Vaex one can do:

```
df = vaex.from_json('./my_data/my_file.json')
```

This is a convenience method which simply wraps `pandas.read_json`, so the same arguments and file reading strategy applies.

### In-memory data representations

One can construct a Vaex DataFrame from a variety of in-memory data representations. Such a common operation is converting a pandas into a Vaex DataFrame:

```
df = vaex.from_pandas(pandas_df, copy_index=True)
```
The `copy_index` argument specifies whether the index column of a pandas DataFrame should be imported into the Vaex DataFrame. Converting a pandas into a Vaex DataFrame is particularly useful since pandas can read data from a large variety of file formats. For instance, we can use pandas to read data from a database, and then pass it to Vaex like so:

```
import vaex
import pandas as pd
import sqlalchemy

connection_string = 'postgresql://readonly:' + 'my_password' + '@my-server.my-company.com:1234/database_name'
engine = sqlalchemy.create_engine(connection_string)

pandas_df = pd.read_sql_query('SELECT * FROM MYTABLE', con=engine)
df = vaex.from_pandas(pandas_df, copy_index=False)
```

Another example is using pandas to read in [SAS](https://www.sas.com/en_us/home.html) files:

```
import vaex
import pandas as pd

pandas_df = pd.read_sas('./my_data/my_file.xport')
df = vaex.from_pandas(pandas_df)
```

One can read in an [arrow table](https://arrow.apache.org/docs/python/generated/pyarrow.Table.html) as a Vaex DataFrame in a similar manner:

```
df = vaex.from_arrow_table(pa_table)
```

Constructing a Vaex DataFrame from numpy arrays can is done like this:

```
import vaex
import numpy as np

x = np.array([1, 2, 3])
y = np.array(['dog', 'cat', 'mouse'])

df = vaex.from_arrays(x=x, y=y)
```

Constructing a DataFrame from a Python dict is also straight-forward:

```
d = {'a': [1, 2, 3], 'b': ['dog', 'cat', 'mouse']}
df = vaex.from_dict(d)
```

At times, one may need to create a single row DataFrame. Vaex has a convenience method which takes individual elements (scalars) and creates the DataFrame:

```
df = vaex.from_scalars(x=5, y='horse')
```

## Data export

One can export Vaex data to multiple file or in-memory data representations:

 - Binary file formats:
 
     - [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format#HDF5)
     - [Apache Arrow](https://arrow.apache.org/)
     - [Apache Parquet](https://parquet.apache.org/)
     - [FITS](https://en.wikipedia.org/wiki/FITS)
     
 - Text based file formats:
 
     - [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
     - [ASCII](https://en.wikipedia.org/wiki/Text_file)
     
 - In-memory data representations:

    - DataFrames:
    
         - [panads](https://pandas.pydata.org/) DataFrame
         - [Apache Arrow](https://arrow.apache.org/) Table
         - [numpy](https://numpy.org/) arrays
         - [Dask](https://dask.org/) arrays
         - Python dictionaries
         - Python items list ( a list of ('column_name', data) tuples)

    - Expressions:
    
         - [panads](https://pandas.pydata.org/) Series
         - [numpy](https://numpy.org/) array
         - [Dask](https://dask.org/) array
         - Python list

### Binary file formats

The most efficient way to store data on disk when you work with Vaex is to use binary file formats. Vaex can export a DataFrame to HDF5, Apache Arrow, Apache Parquet and FITS:

```
df.export_hdf5('./my_output_data/data.hdf5')
df.export_arrow('./my_output_data/data.arrow')
df.export_parquet('./my_output_data/data.parquet')
df.export_fits('./my_output_data/data.fits')
```

Alternatively, one can simply use

```
df.export('./my_output_data/data.hdf5')
df.export('./my_output_data/data.arrow')
df.export('./my_output_data/data.parquet')
df.export('./my_output_data/data.fits')
```

where Vaex will determine the file format of the based on the specified extension of the file name. If the extension is not recognized, an exception will be raised. 

If your data is large, i.e. larger than the available RAM, we recomment exporting to HDF5. 

### Text based file format

At times, it may be useful to export the data to disk in a text based file format such as CSV. In that case one can simply do:

```
df.export_csv('./my_output_data/data.csv')  # `chunk_size` has a default value of 1_000_000
```

The `df.export_csv` method is using `pandas_df.to_csv` behind the scenes, and thus one can pass any argument to `df.export_csv` as would to `pandas_df.to_csv`. The data is exported in chunks and the size of those chunks can be specified by the `chunk_size` argument in `df.export_csv`. In this way, data that is too large to fit in RAM can be saved to disk.

### In memory data representation

Python has a rich ecosystem comprised of various libraries for data manipulation, that offer different functionality. Thus, it is often useful to be able to pass data from one library to another. Vaex is able to pass on its data to other libraries via a number of in-memory representations.

#### DataFrame representations

A vaex DataFrame can be converted to a pandas DataFrame like so:

```
pandas_df = df.to_pandas_df()
```

For DataFrames that are too large to fit in memory, one can specify the `chunk_size` argument, and the `df.to_pandas_df(chunk_size=100_000)` returns an generator yileding a pandas DataFrame with as many rows as indicated by the `chunk_size` argument.

Converting a Vaex DataFrame into an arrow table is similar:

```
# Get a single arrow table
table = df.to_arrow_table()

# Create a generator, yielding an arrow table with the specified number of rows
gen = df.to_arrow_table(chunk_size=1_000_000)
```

A Vaex DataFrame can be lazily exposed as a Dask array:

```
ddf = df.to_dask_array()
```

Keeping it close to pure Python, one can export a Vaex DataFrame as a dictionary:
```
# Get a single Python dict
d_dict = df.to_dict()

# Create a generator, yielding a Python dict with the specified number of rows
gen = df.to_dict(chunk_size=1_000_000)
```

By specifying the `array_type` argument, one can choose whether the data will be represented by numpy arrays, xarrays, or Python lists.

Alternatively, one can also convert a DataFrame to a list of tuples, were the first element of the tuple is the column name, while the second element is the array representation of the data.

```
# Get a single item list
items = df.to_items()

# Create a generator, yielding an item list with the specified number of rows
gen = df.to_items(chunk_size=1_000_000)
```

One can also export a DataFrame as a list of arrays (numpy, xarrays, or Python lists):
```
# Get a single list of arrays
arrays = df.to_arrays(array_type=None)   # by default, array_type=None returns numpy arrays

# Create a generator, yielding an list of lists with the specified number of rows
gen = df.to_items(chunk_size=1_000_000, array_type='list')  # 
```

#### Expression representations

A single vaex Expression can be also converted to a variety of in-memory representations:

```
# pandas Series
x_series = df.x.to_pandas_series()

# numpy array
x_numpy = df.x.to_numpy()

# Dask array
x_dask_array = df.x.to_dask_array()

# Python list
x_list = df.x.tolist()
```

## Examples of in-memory input and output

### DataFrames

In [1]:
import vaex
import numpy as np

Let us construct a DataFrame via a couple of in-memory data representations:

In [2]:
# Construct a DataFrame from Numpy arrays
x = np.arange(2)
y = np.array([10, 20])
z = np.array(['dog', 'cat'])


df_1 = vaex.from_arrays(x=x, y=y, z=z)
df_1

#,x,y,z
0,0,10,dog
1,1,20,cat


In [3]:
# Construct a DataFrame from Python dictionary
data_dict = dict(x=[2, 3], y=[30, 40], z=['cow', 'horse'])

df_2 = vaex.from_dict(data_dict)
df_2

#,x,y,z
0,2,30,cow
1,3,40,horse


In [4]:
# Construct a single row DataFrame
df_3 = vaex.from_scalars(x=4, y=50, z='mouse')
df_3

#,x,y,z
0,4,50,mouse


Now let us concatenate all the DataFrames above:

In [5]:
df = vaex.concat([df_1, df_2, df_3])
df

#,x,y,z
0,0,10,dog
1,1,20,cat
2,2,30,cow
3,3,40,horse
4,4,50,mouse


Once we have the data into a Vaex DataFrame, it is quite easy to pass it on to other common in-memory representations, either in full or in chunks, so that memory issues can be avoided for as long as possible.

In [6]:
pandas_df = df.to_pandas_df()
pandas_df  # looks the same doesn't it

,x,y,z
0,0,10,dog
1,1,20,cat
2,2,30,cow
3,3,40,horse
4,4,50,mouse


In [7]:
arrow_table = df.to_arrow_table()
arrow_table

pyarrow.Table
x: int64
y: int64
z: string

In [8]:
arrays = df.to_arrays()
arrays

[array([0, 1, 2, 3, 4]),
 array([10, 20, 30, 40, 50]),
 array(['dog', 'cat', 'cow', 'horse', 'mouse'], dtype=object)]

In [9]:
my_dict = df.to_dict()
my_dict

{'x': array([0, 1, 2, 3, 4]),
 'y': array([10, 20, 30, 40, 50]),
 'z': array(['dog', 'cat', 'cow', 'horse', 'mouse'], dtype=object)}

In [10]:
items = df.to_items()
items

[('x', array([0, 1, 2, 3, 4])),
 ('y', array([10, 20, 30, 40, 50])),
 ('z', array(['dog', 'cat', 'cow', 'horse', 'mouse'], dtype=object))]

By default, the above methods `.to_arrays`, `.to_dict` and `to.items` return the data as numpy arrays. By specifying the `array_type` keyword argument in each of these 3 methods, we can choose whether to output the data as an xarray object, a standard Python list, or explicitly choose the numpy representation.

In [11]:
# as xarray
arrays = df.to_arrays(array_type='xarray')
arrays

[<xarray.DataArray (dim_0: 5)>
 array([0, 1, 2, 3, 4])
 Dimensions without coordinates: dim_0, <xarray.DataArray (dim_0: 5)>
 array([10, 20, 30, 40, 50])
 Dimensions without coordinates: dim_0, <xarray.DataArray (dim_0: 5)>
 array(['dog', 'cat', 'cow', 'horse', 'mouse'], dtype=object)
 Dimensions without coordinates: dim_0]

In [12]:
# as a Python list
arrays = df.to_arrays(array_type='list')
arrays

[[0, 1, 2, 3, 4],
 [10, 20, 30, 40, 50],
 ['dog', 'cat', 'cow', 'horse', 'mouse']]

In [13]:
# explicity choose numpy (same as the default None)
arrays = df.to_arrays(array_type='numpy')
arrays

[array([0, 1, 2, 3, 4]),
 array([10, 20, 30, 40, 50]),
 array(['dog', 'cat', 'cow', 'horse', 'mouse'], dtype=object)]

For DataFrames that are too large to fit into the available RAM, the above methods all support the `chunk_size` keyword argument. When specified, the methods return a generator, which in turn yields a chunk of data, in the specified format. The generator also yields the row number of the first and the last element of that chunk.

In [14]:
gen_pandas =  df.to_pandas_df(chunk_size=3)
for i1, i2, chunk in gen_pandas:
    print(i1, i2, chunk)
    print()

0 3    x   y    z
0  0  10  dog
1  1  20  cat
2  2  30  cow

3 5    x   y      z
0  3  40  horse
1  4  50  mouse



In [15]:
gen_array = df.to_arrays(chunk_size=3)
for i1, i2, chunk in gen_array:
    print(i1, i2, chunk)
    print()

0 3 [array([0, 1, 2]), array([10, 20, 30]), array(['dog', 'cat', 'cow'], dtype=object)]

3 5 [array([3, 4]), array([40, 50]), array(['horse', 'mouse'], dtype=object)]



In [16]:
gen_dict = df.to_dict(chunk_size=3, array_type='list')
for i1, i2, chunk in gen_dict:
    print(i1, i2, chunk)
    print()

0 3 {'x': [0, 1, 2], 'y': [10, 20, 30], 'z': ['dog', 'cat', 'cow']}

3 5 {'x': [3, 4], 'y': [40, 50], 'z': ['horse', 'mouse']}



One can also lazily pass a vaex DataFrame to dask arrays, so in principle no extra memory should be used:

In [17]:
dask_arrays = df[['x', 'y']].to_dask_array()   # String support coming soon
dask_arrays

dask.array<vaex-df-e5732a06-8942-11ea-9871, shape=(5, 2), dtype=int64, chunksize=(5, 2), chunktype=numpy.ndarray>

### Expressions

One can easily get in-memory representations of Vaex Expressions as well

In [18]:
# Pandas Series
x_series = df.x.to_pandas_series()
x_series

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [19]:
# Numpy array
x_numpy = df.x.to_numpy()
x_numpy

array([0, 1, 2, 3, 4])

In [20]:
# Python list
x_list = df.x.tolist()
x_list

[0, 1, 2, 3, 4]

In [21]:
# Dask array - Lazily passed
x_dask_array = df.x.to_dask_array()
x_dask_array


dask.array<vaex-expression-e578e338-8942-11ea-9871, shape=(5,), dtype=int64, chunksize=(5,), chunktype=numpy.ndarray>